In [ ]:
pip install tensorflow keras matplotlib scikit-learn


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
from collections import Counter

# Define the dataset directory and corrected folder names
dataset_dir = '/content/drive/My Drive/ml_data[processed]/'
class_dirs = {'Fproccessed': 0, 'Rprocessed': 1}  # Corrected folder name for fake images

# Parameters
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Load images and labels dynamically
images, labels = [], []

for folder_name, label in class_dirs.items():
    folder_path = os.path.join(dataset_dir, folder_name)
    if os.path.isdir(folder_path):
        print(f"Processing folder: {folder_name}")
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            try:
                img = load_img(img_path, target_size=IMG_SIZE)
                img_array = img_to_array(img) / 255.0
                images.append(img_array)
                labels.append(label)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    else:
        print(f"Folder not found: {folder_path}")

# Convert to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Print dataset statistics
print(f"Number of images loaded: {len(images)}")
print("Class distribution:", Counter(labels))

# Split into training, validation, and testing datasets
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Data augmentation setup
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixels
    rotation_range=90,
    width_shift_range=0.5,
    height_shift_range=0.5,
    shear_range=0.5,
    zoom_range=0.5,
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect'
)

val_datagen = ImageDataGenerator()

test_datagen = ImageDataGenerator()

# Create data generators
train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)
val_generator = val_datagen.flow(X_val, y_val, batch_size=BATCH_SIZE)
test_generator = test_datagen.flow(X_test, y_test, batch_size=BATCH_SIZE)

# Print dataset size after augmentation
steps_per_epoch = len(train_generator) * BATCH_SIZE
print(f"Number of images after augmentation per epoch: {steps_per_epoch}")

# Define the model architecture
model = Sequential([
    Input(shape=(128, 128, 3)),

    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    Conv2D(256, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.5),

    Conv2D(512, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),

    Conv2D(1024, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),

    Flatten(),
    Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),
    Dropout(0.6),

    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),  # Increased learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Define callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
]

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,  # Maximum epochs
    callbacks=callbacks,
    verbose=1
)

# Evaluate the model on training data
def evaluate_generator(generator, model):
    results = model.evaluate(generator)
    return results[1]  # Return accuracy

train_accuracy = evaluate_generator(train_generator, model)
test_loss, test_accuracy = model.evaluate(test_generator)

# Print results
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


Processing folder: Fproccessed
Processing folder: Rprocessed
Number of images loaded: 598
Class distribution: Counter({1: 316, 0: 282})
Number of images after augmentation per epoch: 448
Epoch 1/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 64s 4s/step - accuracy: 0.5441 - loss: 29.0002 - val_accuracy: 0.5000 - val_loss: 27.6515
Epoch 2/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - accuracy: 0.5253 - loss: 28.1809 - val_accuracy: 0.5000 - val_loss: 27.8544
Epoch 3/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 58s 4s/step - accuracy: 0.5051 - loss: 27.7509 - val_accuracy: 0.5000 - val_loss: 28.0691
Epoch 4/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 0.5524 - loss: 26.9912 - val_accuracy: 0.5000 - val_loss: 27.0302
Epoch 5/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 57s 4s/step - accuracy: 0.5582 - loss: 26.0261 - val_accuracy: 0.5000 - val_loss: 25.0750
Epoch 6/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 57s 4s/step - accuracy: 0.4679 - loss: 25.3053 - val_accuracy: 0.5000 - val_loss: 26.8569
Epoch 7/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/